## Columbia University

This script serves as a basic tutorial for extracting courses of interest from a university. This is by no means the only (or even best way) to go about this process—so if you come up with a process that works better, feel free to implement! If you're unfamiliar with any of the libraries, the comments below annotate reasoning behind each.

In [137]:
import pandas as pd
import numpy as np
import time
import re
import urllib.request #handles urls
from urllib.request import urlopen
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

The first thing we want to do is set up a function for standard preprocessing. It's also useful to list all of the URLs we'll need to send requests to before scraping. We want all courses within a 2 year *academic* calendar (as opposed to an annual calendar). 

In [138]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

#course catalog URLs - 2 academic years 
#only 2019 is available, Fall(3), Summer(2), Spring(1)
# urls array

urls = [{"term": 'Fall 2019', "url":"?site=Directory_of_Classes&instr=&days=&semes=20191&hour="},
        {"term": 'Summer 2019', "url":'?si?site=Directory_of_Classes&instr=&days=&semes=20192&hour='},
        {"term": 'Spring 2019', "url":'?site=Directory_of_Classes&instr=&days=&semes=20193&hour='}]

link = 'https://doc.search.columbia.edu/classes/'

Next, we'll want to import our keyword csv, split our keyword lists, and preprocess them. The way the csv is set up, we'll want to split the words that are indicated as technical (`T`) or normative (`N`) and that we've chosen to include (`Y`). You'll notice that preprocessing is useful for some of our words but not for others. Here, we've chosen to manually alter words that are not usefully preprocessed. In this case, it means replacing instances of words that are stemmed to end in i.

[regex is a bitch here]

In [139]:
#import keywords
keywords = pd.read_csv("../keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


The process behind extracting relevant courses works in two steps:
1. First, we want to find and extract all courses that contain any instance of a normative keyword.
2. Then, we want search within these courses to see if it also contains a technical keyword.

We initialize a data frame with columns for all of the course items we want to extract. It probably makes the most sense to standardize these feature names across all university scripts so that they're easier to merge in the final compiled dataset for all universities. Our items of interest are:
* The course title: `title`
* The department and course number: `dept_num`
* The course description: `description`
* The number of credits for the course: `credits`
* The course instructor: `instructor`
* The link to the course syllabus (if applicable): `syllabus`
* The university the course is extracted from: `university`
* The term that the course is offered during (fall, spring, summer / year): `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [140]:
#init dfs
# columbia = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
#                                 'syllabus','university','term','keyword'])
columbia = pd.DataFrame(columns=['title','university','term','keyword'])

The loop below executes part 1 of our extraction. It's long and kind of messy (sorry), so feel free to play around with the structure if you'd like. The key tasks here are to extract our items of interest based on our search queries and append them to our data frame.

In [141]:
#roster search for all urls
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

for url in urls:
    print("term", url["term"])
    
    #loop through all normative words and extract relevant elements 
    for word in normative: 
        url_keyword = link + word + url['url'] #NOTE:this structure will likely be different between rosters!
        driver = webdriver.Chrome()
        driver.get(url_keyword)
        time.sleep(3)

        #the number of reponses
        elements = driver.find_elements_by_xpath('//*[@id="gsa-search-results"]/li')
        results = len(elements)
        print("elements", len(elements))
        
        
        #scraping each results
        for x in range(0, results):
            print('-------------------')
            print('x', x)
            
#             if x > 0:
#                 driver.get(url_keyword)
#                 time.sleep(3)
                
#             ActionChains(driver).move_to_element(driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/h3/a')).perform()
            
            title = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/h3/a').text
            section = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/h3/a/span').text
            title = title.replace(section, '').strip()
            
    
#             print("SECTION", section)
            
            course_link = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/div[2]').text
#             print('course_link', course_link)
            
            #course page
            #course_link.click()
#             driver.get(course_link)
#             time.sleep(3)
            
#             try:
#                 dept_nums = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[10]/td[2]').text
# #             print('dept_nums', dept_nums)
#             except:
#                 pass
        
#             descs = ''
            
#             try:
#                 credit = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[4]/td[2]').text
#             except:
#                 pass
            
#             try:
#                 profs = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[7]/td[2]').text
#             except:
#                 pass
            
            syllabi = ''  
            uni = 'columbia university'
            term = url["term"]  
            keyword = word
            
#             columbia.loc[x] = [title, dept_nums, descs, credit, profs, syllabi, uni, term, keyword]
            columbia.loc[x] = [title, uni, term, keyword]
#             print('columbia', columbia)
        
        driver.close()


elements 54
-------------------
x 0
TITLE Accounting I: Financial Accoun
-------------------
x 1
TITLE Accounting I: Financial Accoun
-------------------
x 2
TITLE Accounting for Value
-------------------
x 3
TITLE Accounting for Consultants
-------------------
x 4
TITLE (PhD) Accounting & Information
-------------------
x 5
TITLE Earnings Quality & Fundamental
-------------------
x 6
TITLE Fundamental Analysis for Inves
-------------------
x 7
TITLE Financial Planning & Analysis
-------------------
x 8
TITLE Earnings Quality & Fundamental
-------------------
x 9
TITLE Financial Statement Analysis a
-------------------
x 10
TITLE Fundamental Analysis for Inves
-------------------
x 11
TITLE Corporate Transactions and Fin
-------------------
x 12
TITLE Research on Investing with Fun
-------------------
x 13
TITLE ACCOUNTING AND FINANCE
-------------------
x 14
TITLE FINANCIAL ACCOUNTING
-------------------
x 15
TITLE FINANCIAL ACCOUNTING
-------------------
x 16
TITLE ACCOUNTING & BUDGE

TITLE MARSEA WORKSHOP II
-------------------
x 90
TITLE PERSPECTVS ON ECONOMIC STUDIES
-------------------
x 91
TITLE FY WRITING: THE AMERICAS
-------------------
x 92
TITLE PUBLIC SPEAKING
-------------------
x 93
TITLE ADVANCED POETRY WRITING II
-------------------
x 94
TITLE SHAKESPEARE II
-------------------
x 95
TITLE JAMES JOYCE
-------------------
x 96
TITLE ALDOUS HUXLEY
-------------------
x 97
TITLE INTRO LATINX LITERATURE
-------------------
x 98
TITLE AFTER NATURE
-------------------
x 99
TITLE LIT & AMERICAN CITIZENSHIP
-------------------
x 100
TITLE FILM NARRATIVE (SEMINAR)
-------------------
x 101
TITLE SPECIAL PROJECT IN WRITING
-------------------
x 102
TITLE MIGRANT MODERNISM
-------------------
x 103
TITLE INTRO TO FILM & MEDIA STUDIES
-------------------
x 104
TITLE TOPICS IN AMERICAN CINEMA
-------------------
x 105
TITLE DIGITAL SERIES
-------------------
x 106
TITLE UNSEEN MASTERWORKS
-------------------
x 107
TITLE NEW CINEMAS IN LATIN AMERICA
----------------

-------------------
x 38
TITLE PROBABILITY
-------------------
x 39
TITLE ACTING ENSEMBLE DIRECTING II
-------------------
x 40
TITLE ELEMENTARY UKRAINIAN II
-------------------
x 41
TITLE RACE. SPACE, URB SCHOOLS
-------------------
x 42
TITLE NEIGHBORHOOD & COMMUNITY DVLP
-------------------
x 43
TITLE INTRO TO WOMEN'S & GENDER STUDIES
-------------------
x 44
TITLE GENDER & POWER IN GLOBAL PERSP
-------------------
x 45
TITLE GENDER & POWER TRANSNATL PERSP
elements 16
-------------------
x 0
TITLE Private Equity, Growth Equity
-------------------
x 1
TITLE Equity Derivatvs/Related Products
-------------------
x 2
TITLE Foundations of Private Equity
-------------------
x 3
TITLE SOCIETY,HLTH EQUITY&HLTH COMMUNICATION
-------------------
x 4
TITLE LAW & EDUC INST: EQUITY ISSUES
-------------------
x 5
TITLE TOPICS IN OPERATION RESEARCH
-------------------
x 6
TITLE ASSET ALLOCATION
-------------------
x 7
TITLE AMERICAN WELLBEING
-------------------
x 8
TITLE Real Estate Project Class

TITLE INTERNATIONAL ECONOMICS
-------------------
x 44
TITLE DEVPT & ENVIRON ECON WORKSHOP
-------------------
x 45
TITLE ECONOMETRICS WORKSHOP
-------------------
x 46
TITLE INDUSTRIAL ORG WORKSHOP
-------------------
x 47
TITLE APPLIED MICROECONOMICS LABOR
-------------------
x 48
TITLE GLOBAL CAPITAL MARKET & INVESTMENT
-------------------
x 49
TITLE AFRICAN-AMERICAN MUSIC
-------------------
x 50
TITLE IMAGE/IDENT-CONTEMP ADVERTISNG
-------------------
x 51
TITLE TOPICS IN THE BLACK EXPERIENCE
-------------------
x 52
TITLE THESIS RESEARCH-GRADUATE
-------------------
x 53
TITLE Introduction to the History of Architect
-------------------
x 54
TITLE ROMAN ART & ARCHITECTURE
-------------------
x 55
TITLE RENAISSANCE IN IMPERIAL SPAIN
-------------------
x 56
TITLE TWENTIETH CENTURY ART
-------------------
x 57
TITLE TWENTIETH CENTURY ART-DISC
-------------------
x 58
TITLE ARTS OF JAPAN
-------------------
x 59
TITLE A HISTORY OF CHINA IN 27 OBJECTS
-------------------
x 60
TITLE P

-------------------
x 183
TITLE C.P. CAVAFY&THE POETICS OF DESIRE
-------------------
x 184
TITLE AESTHETICS & PHIL OF HIST(ENG)
-------------------
x 185
TITLE 21ST-CENTURY TRANSNATIONAL CINEMA
-------------------
x 186
TITLE PIRANDELLO & MOD ITALIAN DRAMA
-------------------
x 187
TITLE INVENTING THE ANCIENTS
-------------------
x 188
TITLE THE CLASSICAL TRADITION
-------------------
x 189
TITLE ARABIC LITERARY PRODUCTION
-------------------
x 190
TITLE DIRECTED READINGS
-------------------
x 191
TITLE DIRECTED READINGS
-------------------
x 192
TITLE DIRECTED READING & RESEARCH I
-------------------
x 193
TITLE BASIC CONCEPTS-POST-FREUD THGT
-------------------
x 194
TITLE THINKING MOTHERS-PHILOSOPHICAL, PSYCHOAN
-------------------
x 195
TITLE PSYCHOANALYSIS AND SOCIAL SCIENCE RESEAR
-------------------
x 196
TITLE BALKAN AS A METAPHOR
-------------------
x 197
TITLE RESP COND OF RES/REL PLCY ISSU
-------------------
x 198
TITLE RESP COND-RES/REL PLCY ISS-DIS
-------------------
x 

...

TITLE CORE CONCEPTS-CHILD/ADOLESCENT TRAUMA
-------------------
x 28
TITLE FEDERAL POLICY
-------------------
x 29
TITLE DATA MINING FOR SOCIAL SCIENCE
-------------------
x 30
TITLE SW PRACTICE WITH CHILDREN
-------------------
x 31
TITLE HUMAN BEHAV/SOC ENVIRONMNT A
-------------------
x 32
TITLE THE HEALTH CARE SYSTEM
-------------------
x 33
TITLE FAMILY & CHILDRENS SERVICES
-------------------
x 34
TITLE AGING: ISSUES, POLICY, RESEARCH
-------------------
x 35
TITLE WORKERS AND THE WORKPLACE
-------------------
x 36
TITLE SCHOOL-BASED SERVICES
-------------------
x 37
TITLE ADVANCED CLINICAL PRACTICE
-------------------
x 38
TITLE HUMAN RESOURCE MANAGEMENT
-------------------
x 39
TITLE MACRO COMMUNITY PRACTICE
-------------------
x 40
TITLE ADV GENERALIST PRACTICE & PROG
-------------------
x 41
TITLE SEMINAR ON POLICY PRACTICE I
-------------------
x 42
TITLE PRAC IN ALCOLSM & CHEML D
-------------------
x 43
TITLE HUMAN SEXUALITY
-------------------
x 44
TITLE SW PRAC & DOMESTI

TITLE MUSEUM ANTHROP:HIST & THEORY
-------------------
x 169
TITLE HISTORIES OF AMERICAN CITIES
-------------------
x 170
TITLE INTRO EAST ASIAN CIV: TIBET
-------------------
x 171
TITLE INTRO TO ISLAMIC CIVILIZATION
-------------------
x 172
TITLE INTRO TO INDIAN CIVILIZATION
-------------------
x 173
TITLE INTRO-COMP APPL-HLTH CRE/BIOMD
-------------------
x 174
TITLE THIRD YEAR CHINESE W
-------------------
x 175
TITLE HISTORY OF CHINESE LANGUAGE
-------------------
x 176
TITLE CAPITAL FACILITY PLANNING/FIN
-------------------
x 177
TITLE INTL CONSTRUCTION MGMT
-------------------
x 178
TITLE ARCH DESIGN, COMPUT&METHOD
-------------------
x 179
TITLE WORLDS OF ALEXANDER THE GREAT
-------------------
x 180
TITLE ARCHAEOL OF ANC EGYPT & NUBIA
-------------------
x 181
TITLE VERNACULAR PALEOGRAPHY
-------------------
x 182
TITLE SUFISM: PRIMARY TEXTS/CONTEXTS
-------------------
x 183
TITLE GLOBAL FREUD
-------------------
x 184
TITLE Poets, Rebels, Exiles: 100 Years of Russ
---------

TITLE FREUD
-------------------
x 4
TITLE WORLD PHILOLOGY
-------------------
x 5
TITLE FASCISM: AESTHETICS & POLITICS
-------------------
x 6
TITLE POLITICS OF PERFORMANCE
-------------------
x 7
TITLE The Hybrid Voice: Comparative Diasporas
-------------------
x 8
TITLE Socialism/Communism: Dilemma of the Past
-------------------
x 9
TITLE BLOOD/LUST: EARLY MOD MEDIT
-------------------
x 10
TITLE INTERNET ECON, ENG & SOCIETY
-------------------
x 11
TITLE SPORT&SOCIETY IN THE AMERICAS
-------------------
x 12
TITLE Pre-Colonial Mesoamerican Societies and
-------------------
x 13
TITLE SOCIETIES/CULTRS: INDIAN OCEAN
-------------------
x 14
TITLE TOPICS IN MUSIC & SOCIETY
-------------------
x 15
TITLE NEUROSCIENCE AND SOCIETY
-------------------
x 16
TITLE SOCIETAL ADAPT TO TERRORISM
-------------------
x 17
TITLE RELIGION IN CHINESE SOCIETY
-------------------
x 18
TITLE PRIN/APPL OF SOCIETY & CULTURE
-------------------
x 19
TITLE COMPUTERS AND SOCIETY
-------------------
x 20
TIT

TITLE FUNCTNS OF A COMPLEX VARIABLE
-------------------
x 37
TITLE HEALTH PHYSICS
-------------------
x 38
TITLE DIAGNOSTIC RADIOL PRACTICUM
-------------------
x 39
TITLE SUPERVISED RESEARCH
-------------------
x 40
TITLE SUPERVISED RESEARCH
-------------------
x 41
TITLE UNDERGRADUATE FIELDWORK
-------------------
x 42
TITLE PRIN OF ULTRASOUND IN MEDICINE
-------------------
x 43
TITLE ADV SCAFFOLD DES/COMPLX TISSUE
-------------------
x 44
TITLE INTERMEDIATE CANTONESE I
-------------------
x 45
TITLE PROCESS & PRODUCT DESIGN I
-------------------
x 46
TITLE UNCERTAIN/RISK-CIVIL INF SYST
-------------------
x 47
TITLE UNCERTAIN/RISK-CIVIL INF SYST
-------------------
x 48
TITLE MANAGING ENG & CONST PROCESSES
-------------------
x 49
TITLE FIELDWORK
-------------------
x 50
TITLE WORLD PHILOLOGY
-------------------
x 51
TITLE COMPUTER NETWORKS
-------------------
x 52
TITLE A BETTER PLANET BY DESIGN
-------------------
x 53
TITLE CLOSING THE CARBON CYCLE
-------------------
x 54
TITLE

-------------------
x 64
TITLE PROTOTYPING URBAN POLICY
-------------------
x 65
TITLE POLICY & LEGAL CNTXT FOR SUMA
-------------------
x 66
TITLE SOCIAL CONTEXTS IMMIGRATN LAW
-------------------
x 67
TITLE GOVERNING THE GLOBAL ECONOMY
-------------------
x 68
TITLE SUSTAINABLE CITIES
-------------------
x 69
TITLE REFUGEES, FORCED MIGR & DISPLACEMENT
-------------------
x 70
TITLE US - EAST ASIA RELATIONS
-------------------
x 71
TITLE RACE, DRUGS, AND INEQUALITY
-------------------
x 72
TITLE TOPICS IN OPERATION RESEARCH
-------------------
x 73
TITLE AMERICAN POLITICAL DEVELOPMENT
-------------------
x 74
TITLE KOREAN POLITICS
-------------------
x 75
TITLE WORKSHOP IN SUSTAINABLE DEVPT
-------------------
x 76
TITLE ADVANCED TUTORIAL
-------------------
x 77
TITLE ECON OF SUSTAINABILITY MGMT
-------------------
x 78
TITLE EARTH INSTITUTE PRACTICUM
-------------------
x 79
TITLE CONTMP PHOTO&RELATED MEDIA:POL EXHIBITNS
-------------------
x 80
TITLE Restitution or Repatriation of 

In [142]:
columbia

,title,university,term,keyword
0,MENTAL HEALTH POLICY,columbia university,Spring 2019,polic
1,CROSS-NATIONAL HEALTH POLICY,columbia university,Spring 2019,polic
2,HLTH DISPARITIES & PUB POLICY,columbia university,Spring 2019,polic
3,TUTORIAL HEALTH POLICY & MANAGEMENT,columbia university,Spring 2019,polic
4,COMPUTING IN CONTEXT: HEALTH POLICY,columbia university,Spring 2019,polic
5,Conservation Policy,columbia university,Spring 2019,polic
6,ISSUES-APPROACHES-HLTH POL&MGT,columbia university,Spring 2019,polic
7,HEALTHCARE ETHICS,columbia university,Spring 2019,polic
8,MIXED METHODS IN HEALTH SYSTEMS RESEARCH,columbia university,Spring 2019,polic
9,"FOOD, PUBLIC HEALTH & PUBLIC POLICY",columbia university,Spring 2019,polic


Now that we've extracted all courses containing a normative keyword of interest, we need to filter our courses to only return titles that contain a normative AND a technical keyword. This is the case for all words except instances of our preprocessed `privac` and `secur`, for which we want to return all courses, even if they don't contain two keywords. To do this, we'll split the courses into two data frames, apply our respective conditions, and then merge them back together. 

In [143]:
exceptions = columbia.loc[(columbia['keyword']=='privac') | (columbia['keyword'] =='secur')]
exceptions

,title,university,term,keyword


In [144]:
#loop through technical keyword list, extract relevant titles
for word in technical:
    df = columbia[columbia['title'].str.contains(word, flags = re.IGNORECASE)]
    df['keyword2'] = word
    
#join keyword cols
df["keyword"] = df["keyword"].map(str) + "," + df["keyword2"]
df = df.drop(columns="keyword2")

df

/Users/Shiying/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,title,university,term,keyword


NOTE: the above cell is likely not the best nor most simple way to execute this step! Feel free to take special liberties here. It's probably wise to pick out a few titles that you know should be returned manually, then check to see if the script is working as desired. 

In [145]:
#combine dfs 
columbia = pd.concat([df, exceptions])
columbia

,title,university,term,keyword


Lastly, we want to export our csv. Ideally, all csv files should be written to the courses directory in our repository. 

In [146]:
#export as csv
columbia.to_csv('../courses/columbia.csv')